--- Day 21: Dirac Dice ---

There's not much to do as you slowly descend to the bottom of the ocean. The submarine computer challenges you to a nice game of Dirac Dice.

This game consists of a single die, two pawns, and a game board with a circular track containing ten spaces marked 1 through 10 clockwise. Each player's starting space is chosen randomly (your puzzle input). Player 1 goes first.

Players take turns moving. On each player's turn, the player rolls the die three times and adds up the results. Then, the player moves their pawn that many times forward around the track (that is, moving clockwise on spaces in order of increasing value, wrapping back around to 1 after 10). So, if a player is on space 7 and they roll 2, 2, and 1, they would move forward 5 times, to spaces 8, 9, 10, 1, and finally stopping on 2.

After each player moves, they increase their score by the value of the space their pawn stopped on. Players' scores start at 0. So, if the first player starts on space 7 and rolls a total of 5, they would stop on space 2 and add 2 to their score (for a total score of 2). The game immediately ends as a win for any player whose score reaches at least 1000.

Since the first game is a practice game, the submarine opens a compartment labeled deterministic dice and a 100-sided die falls out. This die always rolls 1 first, then 2, then 3, and so on up to 100, after which it starts over at 1 again. Play using this die.

For example, given these starting positions:

Player 1 starting position: 4
Player 2 starting position: 8

This is how the game would go:

-    Player 1 rolls 1+2+3 and moves to space 10 for a total score of 10.
-    Player 2 rolls 4+5+6 and moves to space 3 for a total score of 3.
-    Player 1 rolls 7+8+9 and moves to space 4 for a total score of 14.
-    Player 2 rolls 10+11+12 and moves to space 6 for a total score of 9.
-    Player 1 rolls 13+14+15 and moves to space 6 for a total score of 20.
-    Player 2 rolls 16+17+18 and moves to space 7 for a total score of 16.
-    Player 1 rolls 19+20+21 and moves to space 6 for a total score of 26.
-    Player 2 rolls 22+23+24 and moves to space 6 for a total score of 22.

...after many turns...

-    Player 2 rolls 82+83+84 and moves to space 6 for a total score of 742.
-    Player 1 rolls 85+86+87 and moves to space 4 for a total score of 990.
-    Player 2 rolls 88+89+90 and moves to space 3 for a total score of 745.
-    Player 1 rolls 91+92+93 and moves to space 10 for a final score, 1000.

Since player 1 has at least 1000 points, player 1 wins and the game ends. At this point, the losing player had 745 points and the die had been rolled a total of 993 times; 745 * 993 = 739785.

Play a practice game using the deterministic 100-sided die. The moment either player wins, what do you get if you multiply the score of the losing player by the number of times the die was rolled during the game?


In [ ]:
let playerStep (start:int) (steps:int) =
    let position = ( (start-1) + steps ) % 10 + 1
    position

playerStep 6 (88+89+90)

3

In [ ]:
let playRoundOne (start1: int) (start2: int) =
    let rec doStep start1 start2 score1 score2 step die first =        
        let mutable newDie = die

        let mutable throw = newDie
        newDie <- newDie % 100 + 1
        throw <- throw + newDie
        newDie <- newDie % 100 + 1
        throw <- throw + newDie
        newDie <- newDie % 100 + 1

        let pos1 = playerStep start1 throw
        let newScore1 = score1 + pos1
        // if step < 20 then
        //     printfn "%s %i -> %i  %i" (if first then "Player1" else "Player2") start1 pos1 newScore1
        if newScore1 >= 1000 then
            printfn "%i %i" score2 (step*3)
            score2 * step * 3
        else
            doStep start2 pos1 score2 newScore1 (step+1) newDie (not(first))
    doStep start1 start2 0 0 1 1 true

playRoundOne 4 8  // 739785

745 993


739785

In [ ]:
playRoundOne 9 6  // 1004670

915 1098


1004670

--- Part Two ---

Now that you're warmed up, it's time to play the real game.

A second compartment opens, this time labeled Dirac dice. Out of it falls a single three-sided die.

As you experiment with the die, you feel a little strange. An informational brochure in the compartment explains that this is a quantum die: when you roll it, the universe splits into multiple copies, one copy for each possible outcome of the die. In this case, rolling the die always splits the universe into three copies: one where the outcome of the roll was 1, one where it was 2, and one where it was 3.

The game is played the same as before, although to prevent things from getting too far out of hand, the game now ends when either player's score reaches at least 21.

Using the same starting positions as in the example above, player 1 wins in 444356092776315 universes, while player 2 merely wins in 341960390180808 universes.

Using your given starting positions, determine every possible outcome. Find the player that wins in more universes; in how many universes does that player win?


In [ ]:
// count possible offsets and probabilities for "steps" throws
let rec simplePossibilities pos steps =
    if steps = 0 then
        [pos]
    else
        let pos1 = (pos-1+1) % 10 + 1
        let pos2 = (pos-1+2) % 10 + 1
        let pos3 = (pos-1+3) % 10 + 1
        (simplePossibilities pos1 (steps-1))
        @
        ( 
            (simplePossibilities pos2 (steps-1) )
            @
            (simplePossibilities pos3 (steps-1))
        )

simplePossibilities 0 3 |> Seq.ofList |> Seq.groupBy id |> Seq.map ( fun (key, items) -> (key, Seq.length items))

index,Item1,Item2
0,3,1
1,4,3
2,5,6
3,6,7
4,7,6
5,8,3
6,9,1


In [ ]:
// For given possibility, score and count, calculate new possibilities
let expandPossibilities (position:int) (score:int) (count:int64) =
    //output of simplePossibilities 
    seq {
        (3, 1)
        (4, 3)
        (5, 6)
        (6, 7)
        (7, 6)
        (8, 3)
        (9, 1)
    }
    |> Seq.map( fun (offset, cnt) -> 
        let newPosition = (offset + position - 1) % 10 + 1
        (
            newPosition,
            score + newPosition,
            count * int64(cnt)
        )
    )
    
expandPossibilities 2 100 10

index,Item1,Item2,Item3
0,5,105,10
1,6,106,30
2,7,107,60
3,8,108,70
4,9,109,60
5,10,110,30
6,1,101,10


In [ ]:
// Expand all possible positions for the player
let expandPlayerOptions (options: seq<int*int*int64>) =
    options
    |> Seq.map( fun (position, score, count) -> expandPossibilities position score count )
    |> Seq.concat
    |> Seq.groupBy ( fun (position, score, count ) -> (position, score) )
    |> Seq.map ( 
        fun ((position, score), items) -> 
            (
                position,
                score,
                ( items |> Seq.sumBy ( fun (_,_,count) -> count ) )
            )  
    )

// expandPlayerOptions (seq { (1,0,1L); (2,0,1L) })
expandPlayerOptions (seq { (8,100,10L); (2,100,1L) })

index,Item1,Item2,Item3
0,1,101,11
1,2,102,30
2,3,103,60
3,4,104,70
4,5,105,61
5,6,106,33
6,7,107,16
7,8,108,7
8,9,109,6
9,10,110,3


In [ ]:
// Split the possibilities between wins and loses
let countWins (options: seq<int*int*int64>) (limit: int) =
    options
    |> Seq.fold (
        fun (under, over) (pos, score, count) ->
            if score < limit then 
                (under+count, over)
            else
                (under, over+count)        
        )
        (0L, 0L)

countWins (seq{ (0, 10, 5L); (0,30, 12L)}) 20


Item1,Item2
5,12


In [ ]:
let playGame2 (start1: int) (start2: int) (limit: int) =
    let rec round (options1: seq<int*int*int64>) (options2: seq<int*int*int64>) (wins1: int64) (wins2: int64) =
        let newOptions1 = expandPlayerOptions options1
        let newOptions2 = expandPlayerOptions options2
        let (p1Under, p1Over) = countWins newOptions1 limit
        let (p2Under, p2Over) = countWins newOptions2 limit
        let p2BeforeThrow = (options2 |> Seq.sumBy (fun (_, _, cnt) -> cnt ) )
        // player 1 immediately wins, so we multiply his win possibilities by all p2 possibilities
        let newWins1 = wins1 + p1Over * p2BeforeThrow
        // player 2 goes second, so we multiply his win possibilities only by p1 lose possibilites after the reound
        let newWins2 = wins2 + p1Under * p2Over 
        if p1Under = 0 || p2Under = 0 then
            (newWins1, newWins2)
        else
            round
                ( newOptions1 |> Seq.filter ( fun (_, score, _) -> score < limit ))
                ( newOptions2 |> Seq.filter ( fun (_, score, _) -> score < limit ))
                newWins1
                newWins2
    
    round 
        (seq{ (start1, 0, 1L) }) 
        (seq{ (start2, 0, 1L) }) 
        0L 
        0L
        
playGame2 4 8 21   // 444356092776315  341960390180808 


Item1,Item2
444356092776315,341960390180808


In [ ]:
playGame2 9 6 21   // 444356092776315  341960390180808 

Item1,Item2
492043106122795,267086464416104
